In [20]:
from PIL import Image
import numpy as np

In [21]:
import warnings
warnings.filterwarnings('ignore')

In [46]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [47]:
def load_and_pad_image(image_path, target_size):
    img = Image.open(image_path)
    img_array = np.array(img)
    pad_height = max(0, target_size[0] - img_array.shape[0])
    pad_width = max(0, target_size[1] - img_array.shape[1])
    pad_top = pad_height // 2
    pad_bottom = pad_height - pad_top
    pad_left = pad_width // 2
    pad_right = pad_width - pad_left
    img_array = np.pad(img_array, ((pad_top, pad_bottom), (pad_left, pad_right), (0, 0)), mode='constant', constant_values=0)
    return img_array

In [48]:
def load_image_pairs_with_padding(blurred_folder, correct_folder, target_size):
    blurred_images = []
    correct_images = []
    for filename in os.listdir(blurred_folder):
        blurred_path = f'{blurred_folder}{filename}'
        correct_path = f'{correct_folder}{filename}'  # zakładamy, że nazwy plików są takie same
        if os.path.exists(correct_path):
            blurred_img = load_and_pad_image(blurred_path, target_size)
            correct_img = load_and_pad_image(correct_path, target_size)
            blurred_images.append(blurred_img)
            correct_images.append(correct_img)
    return np.array(blurred_images), np.array(correct_images)

In [49]:
import os

In [50]:
# Ścieżki do folderów
correct_images_folder = 'Pictures/'
blurred_images_folder = 'Zgaussowane/'

target_size = (248, 248)

# Załaduj pary obrazów (rozmyte i nierozmyte)
blurred_images, correct_images = load_image_pairs_with_padding(blurred_images_folder, correct_images_folder, target_size)


AttributeError: module 'PIL.Image' has no attribute 'ANTIALIAS'

In [28]:
blurred_images[2,120,4500]

array([0, 0, 0], dtype=uint8)

In [29]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D

In [30]:
input_img = Input(shape=(target_size[0], target_size[1], 3))

In [41]:
# Encoder
x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

In [42]:
# Model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [43]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(blurred_images, correct_images, test_size=0.2, random_state=42)

In [44]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [45]:
autoencoder.fit(x_train, y_train, epochs=50, batch_size=16, shuffle=True, steps_per_epoch=100, validation_data=(x_test, y_test))

Epoch 1/50


2024-06-04 21:45:41.327224: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at conv_ops_impl.h:679 : RESOURCE_EXHAUSTED: OOM when allocating tensor with shape[2,6000,5500,64] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
2024-06-04 21:45:41.332438: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: RESOURCE_EXHAUSTED: OOM when allocating tensor with shape[2,6000,5500,64] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{function_node __inference_one_step_on_data_7176}}{{node functional_15_1/conv2d_15_1/convolution}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.



ResourceExhaustedError: Graph execution error:

Detected at node functional_15_1/conv2d_15_1/convolution defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/user/anaconda3/envs/pureAI/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/user/anaconda3/envs/pureAI/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/user/anaconda3/envs/pureAI/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/user/anaconda3/envs/pureAI/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/user/anaconda3/envs/pureAI/lib/python3.12/asyncio/base_events.py", line 618, in run_forever

  File "/home/user/anaconda3/envs/pureAI/lib/python3.12/asyncio/base_events.py", line 1951, in _run_once

  File "/home/user/anaconda3/envs/pureAI/lib/python3.